**Description**

The SI-STSAR-7 training dataset with only HH band extracted with *"1- img_extraction.ipynb"* was loaded from the manually edited folder *"only1img4allHHHc5 - Copy"*.

The data was then split 80:20 train-test to train the models.

ResNet, DenseNet, EfficientNet and VGG16 models were tested to see which two are most efficient to continue with.

*Additional information*: With the ratio of 80:20, *"ResNet"* and *"EfficientNet"* models with the best results in terms of speed and accuracy were selected for further training "3- split-70-15-15 (best one).ipynb".





# Dataset and initializations

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import resnet18
from PIL import Image
import os
import numpy as np
import tifffile as tiff


In [ ]:
# SI-STSAR-7 Dataset
class SARImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir #initializing dataset by reading the root directory
        self.transform = transform #the transformations to be applied to the images
        self.classes = sorted(os.listdir(root_dir))  #sorts classes
        self.samples = [] #lists for corresponding labels
        #loop for each class folder in the root directory
        for label, class_dir in enumerate(self.classes):
            class_path = os.path.join(root_dir, class_dir)
            #loop for each image in the class folder
            for fname in os.listdir(class_path):
                if fname.endswith(('tif')):
                    self.samples.append((os.path.join(class_path, fname), label))

    def __len__(self):
        return len(self.samples) #returns the total number of images

    def __getitem__(self, idx):
        img_path, label = self.samples[idx] #image and label at the specified index
        img = tiff.imread(img_path) #load tif image
        img = Image.fromarray(img) #convert tif image to PIL image

        if self.transform:
            img = self.transform(img) #apply the specified transformations to the image
        return img, label


In [ ]:
# dataset direction
data_dir = 'only1img4allHHc5 - Copy'

# Data augmentation
data_augmentation = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    #transforms.RandomRotation(10)
])

# Preprocessing
preprocess_transform = transforms.Compose([
    transforms.ToTensor(),
])

# Applying data augmentation and preprocessing to only training dataset
train_transform = transforms.Compose([
    data_augmentation,
    preprocess_transform
])

# Load the dataset
full_dataset = SARImageDataset(root_dir=data_dir, transform=preprocess_transform)

# Perform a 80-20 train-test split on the original dataset length
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size]) # split the datasets

#data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

# Print the sizes of the datasets
print(f"Total number of samples in the full dataset: {len(full_dataset)}")
print(f"Number of samples in the training set: {len(train_dataset)}")
print(f"Number of samples in the testing set: {len(test_dataset)}")

# print the classes
def print_class(root_dir):
    class_names = sorted(os.listdir(root_dir)) #sorts and lists all folders in the main folder
    for i, class_name in enumerate(class_names): #prints each class with folder names
        print(f"Class {i}: {class_name}")
print_class(data_dir)


Total number of samples in the full dataset: 164564
Number of samples in the training set: 131651
Number of samples in the testing set: 32913
Class 0: GI_6s - GWI_6s - NI_6s
Class 1: MediumFI_6s
Class 2: OW_6s
Class 3: ThickFI_6s
Class 4: ThinFI_6s


In [ ]:
# Training and Testing
from tqdm import tqdm #for time counting

def train(train_loader, model, criterion, optimizer, num_epochs, device):
    model.to(device)

    for epoch in range(num_epochs):
        model.train() #set model for training mode
        running_loss = 0.0 #initializing running loss
        correct = 0 #for time counting
        total = 0 #for time counting
        #progress bar for time counting
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}")
        for i, (images, labels) in progress_bar:
            images, labels = images.to(device), labels.to(device)  #data move to the appropriate device/GPU
            optimizer.zero_grad() #all gradients to zero
            outputs = model(images)
            loss = criterion(outputs, labels) #calculate loss
            loss.backward() #computing the gradient of loss with respect to each parameters in model
            optimizer.step() #updating model

            running_loss += loss.item() * images.size(0) #accumulating loss

            #time counting
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            progress_bar.set_postfix(loss=running_loss/(i+1), accuracy=100.*correct/total)
        epoch_accuracy = 100. * correct / total #accuracy per epoch

        epoch_loss = running_loss / len(train_loader.dataset) #avg loss computation
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.2f}, Accuracy: {epoch_accuracy:.2f}%')

    print("Training complete")
    return model

def test(test_loader, model, device):
    model.to(device)
    model.eval() #set evaluation mode
    correct = 0 #correct predictions
    total = 0 #total samples

    with torch.no_grad():  #iterating over the test dataset without tracking gradients
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  #move data to the appropriate device

            outputs = model(images)  #forward pass

            _, predicted = torch.max(outputs, 1)  #get the index of the max log-probability (prediction)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy %: {accuracy * 100:.2f}%')
    return accuracy

#define the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #for "RuntimeError: Found no NVIDIA driver on your system." error

# ResNet

In [ ]:
#trying ResNet model
from torchvision.models import resnet18

# Load the pre-trained ResNet18 model
model = resnet18(pretrained=True) #model weights pre-trained on the ImageNet dataset, "True" gave better results than "False", about 1-2%

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

C:\Users\co1st\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Redefine first layer (single-band input)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) #RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[32, 1, 32, 32] to have 3 channels, but got 1 channels instead
#the data does not have 3 channel, it's single band HH. so first value should be "1"

# Disable gradients from all layers of the model
for param in model.parameters():
    param.requires_grad = False

# replace the last layer with a new linear layer where gradients are active
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 5) #5 classes

# enable gradients for the new layer
for param in model.fc.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

C:\Users\co1st\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\co1st\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
num_epochs = 10

# turning on gradient calculation for all layers
for param in model.parameters():
    param.requires_grad = True

#the optimizer to optimize all parameters
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# train the model for fine-tuning
fine_tuned_model = train(train_loader, model, criterion, optimizer, num_epochs, device)

# test the model for fine-tuning
fine_tuned_accuracy = test(test_loader, fine_tuned_model, device)

Epoch 1/10: 100%|████████████████████████████████████████| 4115/4115 [07:07<00:00,  9.63it/s, accuracy=58.5, loss=32.8]


Epoch 1/10, Loss: 1.02, Accuracy: 58.53%


Epoch 2/10: 100%|████████████████████████████████████████| 4115/4115 [05:08<00:00, 13.33it/s, accuracy=60.1, loss=31.2]


Epoch 2/10, Loss: 0.98, Accuracy: 60.06%


Epoch 3/10: 100%|████████████████████████████████████████| 4115/4115 [05:17<00:00, 12.97it/s, accuracy=60.7, loss=30.6]


...

Epoch 8/10, Loss: 0.90, Accuracy: 62.96%


Epoch 9/10: 100%|████████████████████████████████████████| 4115/4115 [05:22<00:00, 12.75it/s, accuracy=63.3, loss=28.5]


Epoch 9/10, Loss: 0.89, Accuracy: 63.33%


Epoch 10/10: 100%|███████████████████████████████████████| 4115/4115 [05:14<00:00, 13.08it/s, accuracy=63.8, loss=28.2]


Epoch 10/10, Loss: 0.88, Accuracy: 63.80%
Training complete
Test Accuracy %: 62.16%


In [ ]:
# saving the fine-tuned model
torch.save(fine_tuned_model.state_dict(), 'fine_HH5_resnet.pth')

# DenseNet

In [ ]:
#trying DenseNet model
from torchvision.models import densenet121

# Load the pre-trained DenseNet121 model
model = densenet121(pretrained=True)

print(model)


C:\Users\co1st\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\co1st\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [ ]:
# Modify the first layer for single-channel input
model.features.conv0 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

# Modify the classifier for the number of output classes
model.classifier = nn.Linear(1024, 5)

#disable some layers -optional
for param in model.features.parameters():
    param.requires_grad = False

###########################################
num_epochs = 10

# turning on gradient calculation for all layers
for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.0001)

fine_tuned_model = train(train_loader, model, criterion, optimizer, num_epochs, device)
fine_tuned_accuracy = test(test_loader, fine_tuned_model, device)

torch.save(fine_tuned_model.state_dict(), 'fine_HH5_densenet.pth')


C:\Users\co1st\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/10: 100%|████████████████████████████████████████| 4115/4115 [13:04<00:00,  5.25it/s, accuracy=58.7, loss=32.5]


Epoch 1/10, Loss: 1.02, Accuracy: 58.66%


Epoch 2/10: 100%|████████████████████████████████████████| 4115/4115 [13:42<00:00,  5.01it/s, accuracy=60.5, loss=30.9]


Epoch 2/10, Loss: 0.96, Accuracy: 60.48%


Epoch 3/10: 100%|████████████████████████████████████████| 4115/4115 [13:10<00:00,  5.21it/s, accuracy=61.3, loss=30.2]


...

Epoch 8/10, Loss: 0.89, Accuracy: 63.52%


Epoch 9/10: 100%|████████████████████████████████████████| 4115/4115 [14:02<00:00,  4.88it/s, accuracy=64.1, loss=27.9]


Epoch 9/10, Loss: 0.87, Accuracy: 64.14%


Epoch 10/10: 100%|███████████████████████████████████████| 4115/4115 [13:57<00:00,  4.91it/s, accuracy=64.5, loss=27.7]


Epoch 10/10, Loss: 0.86, Accuracy: 64.54%
Training complete
Test Accuracy %: 60.11%


# EfficientNet

In [ ]:
#trying EfficientNet model
from torchvision.models import efficientnet_b0

# Load the pre-trained EfficientNet-B0 model
model = efficientnet_b0(pretrained=True)

print(model)


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

C:\Users\co1st\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Modify the first layer for single-channel input
model.features[0][0] = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

# Modify the classifier for the number of output classes
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 5)

#disable some layers -optional
for param in model.features.parameters():
    param.requires_grad = False

###########################################
num_epochs = 10

# turning on gradient calculation for all layers
for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.0001)

fine_tuned_model = train(train_loader, model, criterion, optimizer, num_epochs, device)
fine_tuned_accuracy = test(test_loader, fine_tuned_model, device)

# saving the fine-tuned model
torch.save(fine_tuned_model.state_dict(), 'fine_HH5_efficientnet.pth')


Epoch 1/10: 100%|████████████████████████████████████████| 4115/4115 [22:25<00:00,  3.06it/s, accuracy=56.6, loss=34.5]


Epoch 1/10, Loss: 1.08, Accuracy: 56.57%


Epoch 2/10: 100%|████████████████████████████████████████| 4115/4115 [22:51<00:00,  3.00it/s, accuracy=59.8, loss=31.9]


Epoch 2/10, Loss: 1.00, Accuracy: 59.83%


Epoch 3/10: 100%|████████████████████████████████████████| 4115/4115 [24:21<00:00,  2.82it/s, accuracy=60.6, loss=31.2]


...

Epoch 8/10, Loss: 0.91, Accuracy: 62.94%


Epoch 9/10: 100%|████████████████████████████████████████| 4115/4115 [24:33<00:00,  2.79it/s, accuracy=63.4, loss=28.7]


Epoch 9/10, Loss: 0.90, Accuracy: 63.38%


Epoch 10/10: 100%|███████████████████████████████████████| 4115/4115 [22:42<00:00,  3.02it/s, accuracy=63.8, loss=28.5]


Epoch 10/10, Loss: 0.89, Accuracy: 63.78%
Training complete
Test Accuracy %: 63.47%


# VGG16

https://medium.com/@ilaslanduzgun/create-vgg-from-scratch-in-pytorch-aa194c269b55

In [ ]:
#trying VGG16 model
from torchvision.models import vgg16

# Load the pre-trained VGG16 model
model = vgg16(pretrained=True)

print(model)


C:\Users\co1st\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# Modify the first layer for single-channel input
model.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

# Modify the classifier for the number of output classes
model.classifier[6] = nn.Linear(4096, 5)

#disable some layers -optional
for param in model.features.parameters():
    param.requires_grad = False

###########################################
num_epochs = 10

# turning on gradient calculation for all layers
for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.0001)

fine_tuned_model = train(train_loader, model, criterion, optimizer, num_epochs, device)
fine_tuned_accuracy = test(test_loader, fine_tuned_model, device)

# saving the fine-tuned model
torch.save(fine_tuned_model.state_dict(), 'fine_HH5_vgg16.pth')


Epoch 1/10: 100%|████████████████████████████████████████| 4115/4115 [47:30<00:00,  1.44it/s, accuracy=55.7, loss=35.1]


Epoch 1/10, Loss: 1.10, Accuracy: 55.70%


Epoch 2/10: 100%|██████████████████████████████████████████| 4115/4115 [46:47<00:00,  1.47it/s, accuracy=58, loss=33.3]


Epoch 2/10, Loss: 1.04, Accuracy: 58.01%


Epoch 3/10: 100%|████████████████████████████████████████| 4115/4115 [46:20<00:00,  1.48it/s, accuracy=59.5, loss=32.1]


...

Epoch 8/10, Loss: 0.96, Accuracy: 60.86%


Epoch 9/10: 100%|████████████████████████████████████████| 4115/4115 [54:19<00:00,  1.26it/s, accuracy=61.1, loss=30.7]


Epoch 9/10, Loss: 0.96, Accuracy: 61.08%


Epoch 10/10: 100%|███████████████████████████████████████| 4115/4115 [48:06<00:00,  1.43it/s, accuracy=61.1, loss=30.6]


Epoch 10/10, Loss: 0.96, Accuracy: 61.07%
Training complete
Test Accuracy %: 60.77%
